# Project

In [22]:
import pandas as pd

ucc_train = pd.read_csv("UCC/train.csv")
aapl = pd.read_csv("data/Apple-Twitter-Sent.csv", encoding="ISO-8859-1")

In [23]:
aapl = aapl.drop(columns="_golden _unit_state _last_judgment_at date id query sentiment_gold".split())
aapl

,_unit_id,_trusted_judgments,sentiment,sentiment:confidence,text
0,623495513,10,3,0.6264,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,623495514,12,3,0.8129,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,623495515,10,3,1.0000,My cat only chews @apple cords. Such an #Apple...
3,623495516,17,3,0.5848,I agree with @jimcramer that the #IndividualIn...
4,623495517,3,3,0.6474,Nobody expects the Spanish Inquisition #AAPL
...,...,...,...,...,...
3881,623499442,13,3,0.7757,(Via FC) Apple Is Warming Up To Social Media -...
3882,623499450,16,3,0.6225,RT @MMLXIV: there is no avocado emoji may I as...
3883,623499486,14,5,0.9347,@marcbulandr I could not agree more. Between @...
3884,623499514,13,1,0.9230,My iPhone 5's photos are no longer downloading...


In [47]:
import nltk
import re
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, TweetTokenizer

tweets = list(zip(aapl["text"], aapl["sentiment"]))

ttk = TweetTokenizer()

tokens = [(ttk.tokenize(tweet), sentiment) for tweet, sentiment in tweets]

filtered = []
for tweet in tokens:
    toks = []
    for tok in tweet[0]:
        if tok.isalpha():
            toks.append(tok)
        if "#" in tok or "@" in tok:
            toks.append(tok)
    filtered.append((toks, tweet[1]))

tagged = [(nltk.pos_tag(tweet), sentiment) for tweet, sentiment in filtered]

tagged[2]

([('My', 'PRP$'),
  ('cat', 'NN'),
  ('only', 'RB'),
  ('chews', 'VBZ'),
  ('@apple', 'JJ'),
  ('cords', 'NNS'),
  ('Such', 'JJ'),
  ('an', 'DT'),
  ('#AppleSnob', 'NN')],
 '3')